# [Advent of Code 2020 Day 14](https://adventofcode.com/2020/day/14)

This looks cracked. More low-level computer stuff.

## Initial setup

In [ ]:
import ipytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()

## Input Parsing

In [ ]:
def parse_input(filename: str) -> list[tuple[str, str, str] | tuple[str, str]]:

    gen = yield_line(filename)
    instructions = []

    for line in gen:
        print(f"parsing '{line}'")
        if line.startswith("mem"):
            print("this starts with mem, going to parse it like that")
            address, value = parse(r"mem\[(\d+)\] = (\d+)", line)
            instructions.append(("mem", address, value))
        elif line.startswith("mask"):
            print("this starts with mask, going to parse it like that")
            mask = parse(r"mask = (.*)", line)[0]
            print(f"received {mask}")
            instructions.append(("mask", mask))

    return instructions

## Part 1
Let's go the straightforward approach, because how else would you do it? First I'm going to make a mask class.

In [ ]:
class Mask:
    def __init__(self, mask: str):
        assert len(mask) == 36
        self.mask: list[str] = list(mask)
    def apply(self, other):
        assert len(other) == 36
        for idx, char in enumerate(other):
            if char == "X":
                continue
            self.mask[idx] = char
    def __repr__(self):
        return "".join(self.mask)

In [ ]:
%%ipytest
def test_mask_creation():
    assert Mask("000000000000000000000000000000001011").mask == list("000000000000000000000000000000001011")

def test_mask_apply_11_73():
    mask = Mask("000000000000000000000000000000001011")
    mask.apply("XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X")
    assert str(mask) == "000000000000000000000000000001001001"

def test_mask_apply_101_101():
    mask = Mask("000000000000000000000000000001100101")
    mask.apply("XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X")
    assert str(mask) == "000000000000000000000000000001100101"

def test_mask_apply_0_64():
    mask = Mask("000000000000000000000000000000000000")
    mask.apply("XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X")
    assert str(mask) == "000000000000000000000000000001000000"

OK, now a helper method to convert int to 36-bit bitmask.

In [ ]:
def int_to_mask(num: int) -> str:
    tmp = bin(num)[2:]
    return ("0" * (36 - len(tmp))) + tmp

In [ ]:
%%ipytest
def test_int_to_mask():
    assert int_to_mask(11) == "000000000000000000000000000000001011"
    assert int_to_mask(73) == "000000000000000000000000000001001001"
    assert int_to_mask(101) == "000000000000000000000000000001100101"
    assert int_to_mask(0) == "000000000000000000000000000000000000"
    assert int_to_mask(64) == "000000000000000000000000000001000000"

In [ ]:
def part_one(data: list[tuple[str, str, str] | tuple[str, str]]) -> int:
    memory = {}
    mask = None

    for item in data:
        if item[0] == "mem":
            assert mask is not None
            address, value = item[1], item[2]
            memory[address] = Mask(int_to_mask(int(value)))
            memory[address].apply(mask)
        elif item[0] == "mask":
            mask = item[1]
        else:
            raise Exception(f"Invalid command {item[0]}")

    return sum([int(str(item), 2) for item in memory.values()])

In [ ]:
%%ipytest
def test_part_one():
    assert part_one(parse_input("example1")) == 165
    assert part_one(parse_input("input")) == 11926135976176

## Part 2
Lorem ipsum

In [ ]:
def part_two(data: Any) -> int | str:
    return 0x3f3f3f3f + 2

In [ ]:
%%ipytest
def test_part_two():
    assert part_two(parse_input("example1")) == 0x3f3f3f3f + 2
    assert part_two(parse_input("input")) == 0x3f3f3f3f + 2